# **NAME: PARTHA SAKHA PAUL**

# **ROLL: MA23M016**

    CS6910_assignment 2


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np

# data_dir = '/content/drive/My Drive/nature_12K'


# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])

# dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# data_dir = '/content/drive/My Drive/nature_12K/inaturalist_12K'

In [2]:
def prepare_data_loaders(data_augment, batch_size=64):
    if data_augment:
        # With data augmentation
        train_transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.450, 0.470, 0.490], std=[0.200, 0.220, 0.240]),
        ])
    else:
        # Without data augmentation
        train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        ])
    
    # Load datasets
    train_data = datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/train', transform=train_transform)
#     validation_dataset = datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/val', transform=val_transform)
#     print(len(train_data))
    # print(train_data[9998])
    # Creating data indices for training and validation splits:
    dataset_size = len(train_data)  #9999
    indices = list(range(dataset_size)) #[0,1,...,9998]
    split = int(np.floor(0.2 * dataset_size))  #1999
#     print(split)
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]  # [1999,...,9998] , [0,...,1998]
    # print((val_indices))

    # Creating data samplers and loaders
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    # print(type(train_sampler))

    train_loader = DataLoader(train_data, batch_size=64, sampler=train_sampler)
    # print(len(train_loader))
    validation_loader = DataLoader(train_data, batch_size=64, sampler=valid_sampler)
    # print(*validation_loader)
    # Prepare data loaders
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#     validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, validation_loader

In [17]:
activation_functions = {
            'relu': nn.ReLU(),
            'gelu': nn.GELU(),
            'silu': nn.SiLU(),
            'mish': nn.Mish()
        }

In [ ]:
# # Defining transforms for the training, validation, and testing sets
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])

# data_dir = '/kaggle/input/inaturalist/inaturalist_12K/train'

# # Load the datasets with ImageFolder
# train_data = datasets.ImageFolder(data_dir, transform=transform)
# print(len(train_data))
# # print(train_data[9998])
# # Creating data indices for training and validation splits:
# dataset_size = len(train_data)  #9999
# indices = list(range(dataset_size)) #[0,1,...,9998]
# split = int(np.floor(0.2 * dataset_size))  #1999
# print(split)
# np.random.shuffle(indices)
# train_indices, val_indices = indices[split:], indices[:split]  # [1999,...,9998] , [0,...,1998]
# # print((val_indices))

# # Creating data samplers and loaders
# train_sampler = SubsetRandomSampler(train_indices)
# valid_sampler = SubsetRandomSampler(val_indices)
# # print(type(train_sampler))

# train_loader = DataLoader(train_data, batch_size=64, sampler=train_sampler)
# # print(len(train_loader))
# validation_loader = DataLoader(train_data, batch_size=64, sampler=valid_sampler)
# # print(*validation_loader)

In [18]:
# defining a class CNN that inherits from nn.Module which is PyTorch's base class for all neural network
class my_CNN(nn.Module):
    def __init__(self, num_classes=10, num_filters=[32, 64, 128, 256, 512], filter_sizes=[3, 3, 3, 3, 3], activation_fn='relu', num_neurons_dense = 512, dropout_rate = 0.5, use_batchnorm='no'):
        super(my_CNN, self).__init__()
        # Initializing class variables
        self.num_classes = num_classes
        self.num_filters = num_filters
        self.filter_sizes = filter_sizes
        self.activation_fn = activation_functions[activation_fn]
        self.num_neurons_dense = num_neurons_dense
        self.dropout_rate = dropout_rate
        # Convert yes/no to boolean
        self.use_batchnorm = True if use_batchnorm.lower() == 'yes' else False

        # Creating convolution layers using ModuleList
        self.conv_layers = nn.ModuleList()
        # List to hold batchnorm layers for conv
        self.batchnorm_layers_conv = nn.ModuleList()
        in_channels = 3  # since, input images are RGB
        for out_channels, kernel_size in zip(num_filters, filter_sizes):
            # a convolution layer with in-out channels and kernel size
            conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, padding=1)
            self.conv_layers.append(conv_layer)
            if self.use_batchnorm:
                # Add a Batchnorm2d layer for each conv layer
                self.batchnorm_layers_conv.append(nn.BatchNorm2d(out_channels))
            # Update in_channels for the next layer
            in_channels = out_channels

            
        # Defining dropout before it's used
        self.dropout = nn.Dropout(dropout_rate)
        # find the size needed to flatten the conv layer outputs, initializing the dense layer accordingly
        self.init_flatten_size(input_shape=(3, 224, 224))  # input images are 224x224 RGB images
        # initializing the dense layer from flatten size
        self.dense = nn.Linear(self.flatten_size, num_neurons_dense)
        if self.use_batchnorm:
             # Batchnorm1d for the dense layer
            self.batchnorm_dense = nn.BatchNorm1d(num_neurons_dense)
        # output layer: maps from the dense layer to the number of classes.
        self.out = nn.Linear(num_neurons_dense, num_classes)


    def init_flatten_size(self, input_shape):
        # to disable gradient calculations for speed up this computation
        with torch.no_grad():
            # a dummy input tensor of the correct shape
            input_tensor = torch.zeros(1, *input_shape)
            # Forward prop through the conv layers to find output size
            output = self.forward_conv_layers(input_tensor)
            # total number of output features is the flat size needed for the dense layer input
            self.flatten_size = output.numel()

    # a function for sequentially passing input through all convolutional layers and applying activation and pooling
    def forward_conv_layers(self, x):
        # Convolution layer
        for i, conv in enumerate(self.conv_layers):
            x = conv(x)  # Convolution operation
            if self.use_batchnorm:
                # Apply batchnorm if enabled
                x = self.batchnorm_layers_conv[i](x)
            x = self.activation_fn(x)  # Apply the specified activation function
            x = F.max_pool2d(x, 2)  # max pooling with a kernel size of 2
            x = self.dropout(x)  # Apply dropout after pooling
        return x

    # forward method defines how the input tensor is transformed through the model
    def forward(self, x):
        # Passing input through the convolution blocks
        x = self.forward_conv_layers(x)
        # Flattening the output for the dense layer
        x = torch.flatten(x, 1)
        # Dense layer
        x = self.dense(x)
        if self.use_batchnorm:
            # Apply batchnorm if enabled to dense layer
            x = self.batchnorm_dense(x)
        x = self.activation_fn(x)
        # Applying dropout before the final layer
        x = self.dropout(x)
        # Output layer
        x = self.out(x)
        return x
    

    # Method for training and evaluating the model
    def train_and_evaluate(self, train_loader, validation_loader, n_epochs=10, lr=0.001, device=None):
        if device is None:
            # setting up CUDA if available, otherwise, CPU
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.to(device)  # transferring the model to the selected device (GPU or CPU)

        # Initializing the loss function and optimizer
        loss_function = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)  # self.parameters() is a built-in method provided by PyTorch's nn.Module that collects all the learnable parameters of our model

        # Training loop
        for epoch in range(n_epochs):
            self.train()  # Setting model to training mode
            training_loss = 0.0
            # tracking number of correctly predicted instances
            correct_train = 0
            # tracking total number of instances
            total_train = 0

            # Iterate over the training data
            for images, labels in train_loader:
                # transferring images and labels to the current device (GPU or CPU)
                images, labels = images.to(device), labels.to(device)
                # clearing the gradients of all optimized variables
                optimizer.zero_grad()
                # Forward prop: computing predicted outputs by passing inputs to the model
                outputs = self(images)
                # calculating the loss between predicted and true labels
                loss = loss_function(outputs, labels)
                # Backward prop: computing gradient of the loss with respect to parameters
                loss.backward()
                # optimization step (parameter update)
                optimizer.step()
                # updating running training loss
                training_loss += loss.item() * images.size(0)
                # converting output probabilities to predicted class
                _, predicted_train = torch.max(outputs.data, 1)
                # updating total number of instances
                total_train += labels.size(0)
                # updating correctly predicted instances
                correct_train += (predicted_train == labels).sum().item()

            # average loss over an epoch
            training_loss /= len(train_loader.sampler)
            # train accuracy
            train_accuracy = correct_train / total_train

            # Evaluation phase
            self.eval()  # Setting model to evaluation mode
            # initializing the validation loss for the epoch
            validation_loss = 0.0
            # tracking number of correctly predicted instances
            correct_val = 0
            # tracking total number of instances
            total_val = 0
            # disable gradient calculation for validation, saving memory and computations
            with torch.no_grad():
                # iterating over the validation data loader
                for images, labels in validation_loader:
                    # transferring images and labels to the current device (GPU or CPU)
                    images, labels = images.to(device), labels.to(device)
                    # Forward prop: computing predicted outputs by passing inputs to the model
                    outputs = self(images)
                    # calculating the loss between predicted and true labels
                    loss = loss_function(outputs, labels)
                    # updating running validation loss
                    validation_loss += loss.item() * images.size(0)
                    # converting output probabilities to predicted class
                    _, predicted = torch.max(outputs.data, 1)
                    # updating total number of instances
                    total_val += labels.size(0)
                    # updating correctly predicted instances
                    correct_val += (predicted == labels).sum().item()

            # average validation loss over an epoch
            validation_loss /= len(validation_loader.sampler)
            # validation accuracy
            val_accuracy = correct_val / total_val

            # printing training and validation results
            print(f'Epoch {epoch+1}, Training Loss: {training_loss:.4f}, Training Accuracy: {train_accuracy*100:.2f}, \nValidation Loss: {validation_loss:.4f}, Validation Accuracy: {val_accuracy*100:.2f}')
            wandb.log({"train_accuracy": train_accuracy * 100, "training_loss": training_loss})
            wandb.log({"val_accuracy": val_accuracy * 100, "val_loss": validation_loss})


# model with parameters
model = my_CNN(num_classes=10,
                  num_filters=[32, 64, 128, 256, 512],
                  filter_sizes=[7,9,7,9,9],
                  activation_fn='relu',
                  num_neurons_dense=512,
                  dropout_rate = 0.5, 
                  use_batchnorm='yes')

print(model)

# model.train_and_evaluate(train_loader, validation_loader, n_epochs=10, lr=0.001)


# References: 
# https://pytorch.org/docs/stable/nn.html
# https://pytorch.org/docs/stable/nn.functional.html
# https://pytorch.org/docs/stable/cuda.html
# https://pytorch.org/docs/stable/data.html
# https://pytorch.org/docs/stable/optim.html

my_CNN(
  (activation_fn): ReLU()
  (conv_layers): ModuleList(
    (0): Conv2d(3, 32, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(32, 64, kernel_size=(9, 9), stride=(1, 1), padding=(1, 1))
    (2): Conv2d(64, 128, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
    (3): Conv2d(128, 256, kernel_size=(9, 9), stride=(1, 1), padding=(1, 1))
    (4): Conv2d(256, 512, kernel_size=(9, 9), stride=(1, 1), padding=(1, 1))
  )
  (batchnorm_layers_conv): ModuleList(
    (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (dense): Linear(in

In [5]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

key = input('Enter your API:')
wandb.login(key=key)  #25c2257eaf6c22aa056893db14da4ee2bf0a531a

Enter your API: 25c2257eaf6c22aa056893db14da4ee2bf0a531a


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
sweep_config = {
    'method': 'random',
    'name' : 'cnn kaggle new 3',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'num_filters': {
            'values': [[128,128,64,64,32], [64, 64, 64, 64, 64], [256, 128, 64, 32, 16], [256,256,128,64,32],[256,128,64,32,16]] 
        },
        'filter_sizes':{
            'values': [[7,7,5,3,3], [7,7,5,5,3], [7,7,7,5,3], [7,7,7,3,3], [5,5,5,3,3], [7,5,7,5,3]]
        },
        'activation_fn': {
            'values': ['relu', 'gelu', 'silu', 'mish']
        },
        'dropout': {
            'values': [0.2, 0.3]
        },
        'batch_norm':{
            'values': ['true','false']
        },
        'data_augment': {
            'values': ['false']
        },
        'num_neurons_dense':{
            'values': [256,128]
        }
    }
}

sweep_id = wandb.sweep(sweep = sweep_config, project="Deep_learning_A2")

Create sweep with ID: wa7l58wm
Sweep URL: https://wandb.ai/parthasakhapaul/Deep_learning_A2/sweeps/wa7l58wm


In [7]:
def main():
    # Initialize a new wandb run
    with wandb.init() as run:
        run_name = "-ac-"+wandb.config.activation_fn+"-filters-"+str(wandb.config.num_filters)+"-filt_size-"+str(wandb.config.filter_sizes)+"-dropout-"+str(wandb.config.dropout)+"-batch_norm-"+wandb.config.batch_norm+"-data_aug-"+wandb.config.data_augment+"-num_neurons_dense-"+str(wandb.config.num_neurons_dense)
        wandb.run.name=run_name
        # Model object creation
        model = my_CNN(num_classes = 10,
                  num_filters = wandb.config.num_filters,
                  filter_sizes = wandb.config.filter_sizes,
                  activation_fn = wandb.config.activation_fn,
                  num_neurons_dense = wandb.config.num_neurons_dense,
                  dropout_rate = wandb.config.dropout,
                  use_batchnorm = wandb.config.batch_norm)
        
        data_augment = wandb.config.data_augment
        train_loader, validation_loader = prepare_data_loaders(data_augment)
        # Model training
        model.train_and_evaluate(train_loader, validation_loader, n_epochs=5, lr=0.001)        

wandb.agent(sweep_id, function=main, count=40)
wandb.finish()

wandb: Agent Starting Run: 2wdptiuy with config:
wandb: 	activation_fn: silu
wandb: 	batch_norm: false
wandb: 	data_augment: false
wandb: 	dropout: 0.2
wandb: 	filter_sizes: [7, 7, 7, 5, 3]
wandb: 	num_filters: [64, 64, 64, 64, 64]
wandb: 	num_neurons_dense: 256
wandb: Currently logged in as: parthasakhapaul. Use `wandb login --relogin` to force relogin


Epoch 1, Training Loss: 2.2389, Training Accuracy: 17.32, 
Validation Loss: 2.1999, Validation Accuracy: 20.16
Epoch 2, Training Loss: 2.1875, Training Accuracy: 19.95, 
Validation Loss: 2.2401, Validation Accuracy: 16.21
Epoch 3, Training Loss: 2.1707, Training Accuracy: 20.85, 
Validation Loss: 2.1395, Validation Accuracy: 21.86
Epoch 4, Training Loss: 2.1609, Training Accuracy: 20.71, 
Validation Loss: 2.1844, Validation Accuracy: 20.36
Epoch 5, Training Loss: 2.1670, Training Accuracy: 20.34, 
Validation Loss: 2.1702, Validation Accuracy: 20.66


train_accuracy,▁▆██▇
training_loss,█▃▂▁▂
val_accuracy,▆▁█▆▇
val_loss,▅█▁▄▃
train_accuracy,20.3375
training_loss,2.16701
val_accuracy,20.66033
val_loss,2.17023


wandb: Agent Starting Run: y65ex3m2 with config:
wandb: 	activation_fn: silu
wandb: 	batch_norm: true
wandb: 	data_augment: false
wandb: 	dropout: 0.3
wandb: 	filter_sizes: [5, 5, 5, 3, 3]
wandb: 	num_filters: [128, 128, 64, 64, 32]
wandb: 	num_neurons_dense: 128


Epoch 1, Training Loss: 2.2298, Training Accuracy: 16.79, 
Validation Loss: 2.1652, Validation Accuracy: 20.06
Epoch 2, Training Loss: 2.1809, Training Accuracy: 20.25, 
Validation Loss: 2.1199, Validation Accuracy: 22.86
Epoch 3, Training Loss: 2.1667, Training Accuracy: 20.85, 
Validation Loss: 2.1314, Validation Accuracy: 21.81
Epoch 4, Training Loss: 2.1481, Training Accuracy: 21.93, 
Validation Loss: 2.1635, Validation Accuracy: 19.96
Epoch 5, Training Loss: 2.1321, Training Accuracy: 22.61, 
Validation Loss: 2.1162, Validation Accuracy: 23.71


...

Epoch 1, Training Loss: 2.2980, Training Accuracy: 11.06, 
Validation Loss: 2.2811, Validation Accuracy: 12.51
Epoch 2, Training Loss: 2.2742, Training Accuracy: 12.60, 
Validation Loss: 2.2866, Validation Accuracy: 12.86
Epoch 3, Training Loss: 2.2588, Training Accuracy: 13.54, 
Validation Loss: 2.2653, Validation Accuracy: 15.11
Epoch 4, Training Loss: 2.2362, Training Accuracy: 16.21, 
Validation Loss: 2.2076, Validation Accuracy: 17.06
Epoch 5, Training Loss: 2.2215, Training Accuracy: 16.80, 
Validation Loss: 2.2066, Validation Accuracy: 18.31


train_accuracy,▁▃▄▇█
training_loss,█▆▄▂▁
val_accuracy,▁▁▄▆█
val_loss,██▆▁▁
train_accuracy,16.8
training_loss,2.22146
val_accuracy,18.30915
val_loss,2.20661


wandb: Agent Starting Run: x6y620d5 with config:
wandb: 	activation_fn: gelu
wandb: 	batch_norm: false
wandb: 	data_augment: false
wandb: 	dropout: 0.2
wandb: 	filter_sizes: [7, 7, 5, 5, 3]
wandb: 	num_filters: [256, 128, 64, 32, 16]
wandb: 	num_neurons_dense: 128


Epoch 1, Training Loss: 2.2476, Training Accuracy: 15.35, 
Validation Loss: 2.2407, Validation Accuracy: 14.96
Epoch 2, Training Loss: 2.2168, Training Accuracy: 17.46, 
Validation Loss: 2.2193, Validation Accuracy: 18.11
Epoch 3, Training Loss: 2.2046, Training Accuracy: 17.77, 
Validation Loss: 2.2319, Validation Accuracy: 17.01
Epoch 4, Training Loss: 2.1934, Training Accuracy: 18.27, 
Validation Loss: 2.1913, Validation Accuracy: 20.66
Epoch 5, Training Loss: 2.1786, Training Accuracy: 19.36, 
Validation Loss: 2.1912, Validation Accuracy: 19.31


train_accuracy,▁▅▅▆█
training_loss,█▅▄▂▁
val_accuracy,▁▅▄█▆
val_loss,█▅▇▁▁
train_accuracy,19.3625
training_loss,2.17861
val_accuracy,19.30965
val_loss,2.19117


In [26]:
def prepare_data_loaders_for_test(data_augment, batch_size=64):
    if data_augment:
        # With data augmentation
        transform_train = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.450, 0.470, 0.490], std=[0.200, 0.220, 0.240]),
        ])
    else:
        # Without data augmentation
        transform_train = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
    
    # Common transform for test/validation dataset
    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    
    # Load datasets
    train_data = datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/train', transform=transform_train)
    test_data = datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/val', transform=transform_test)  # Use 'val_data' for clarity
    
    # Creating data loaders directly without splitting since val_data is already a separate dataset
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)  # Shuffle instead of using a sampler for the entire dataset
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)  # No need to shuffle validation data

    return train_loader, test_loader


In [27]:
# ac-gelu-filters-[64, 64, 64, 64, 64]-filt_size-[3, 5, 7, 3, 3]-dropout-0.2-num_neurons_dense-64
sweep_config = {
    'method': 'bayes',
    'name' : 'cnn kaggle test 18.15',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'num_filters': {
            'values': [[64, 64, 64, 64, 64]] 
        },
        'filter_sizes':{
            'values': [[3,5,7,3,3]]
        },
        'activation_fn': {
            'values': ['gelu']
        },
        'dropout': {
            'values': [0.2]
        },
        'batch_norm':{
            'values': ['false']
        },
        'data_augment': {
            'values': ['false']
        },
        'num_neurons_dense':{
            'values': [64]
        }
    }
}

sweep_id = wandb.sweep(sweep = sweep_config, project="Deep_learning_A2")

Create sweep with ID: l43gicl5
Sweep URL: https://wandb.ai/parthasakhapaul/Deep_learning_A2/sweeps/l43gicl5


In [28]:
def main():
    # Initialize a new wandb run
    with wandb.init() as run:
        run_name = "-ac-"+wandb.config.activation_fn+"-filters-"+str(wandb.config.num_filters)+"-filt_size-"+str(wandb.config.filter_sizes)+"-dropout-"+str(wandb.config.dropout)+"-batch_norm-"+wandb.config.batch_norm+"-data_aug-"+wandb.config.data_augment+"-num_neurons_dense-"+str(wandb.config.num_neurons_dense)
        wandb.run.name=run_name
        # Model object creation
        model = my_CNN(num_classes = 10,
                  num_filters = wandb.config.num_filters,
                  filter_sizes = wandb.config.filter_sizes,
                  activation_fn = wandb.config.activation_fn,
                  num_neurons_dense = wandb.config.num_neurons_dense,
                  dropout_rate = wandb.config.dropout,
                  use_batchnorm = wandb.config.batch_norm)
        
        data_augment = wandb.config.data_augment
        train_loader, test_loader = prepare_data_loaders_for_test(data_augment)
        # Model training
        model.train_and_evaluate(train_loader, test_loader, n_epochs=10, lr=0.001)        

wandb.agent(sweep_id, function=main, count=1)  
wandb.finish()

wandb: Agent Starting Run: h32nd1aj with config:
wandb: 	activation_fn: gelu
wandb: 	batch_norm: false
wandb: 	data_augment: false
wandb: 	dropout: 0.2
wandb: 	filter_sizes: [3, 5, 7, 3, 3]
wandb: 	num_filters: [64, 64, 64, 64, 64]
wandb: 	num_neurons_dense: 64


Epoch 1, Training Loss: 2.2255, Training Accuracy: 17.28, 
Validation Loss: 2.3505, Validation Accuracy: 11.65
Epoch 2, Training Loss: 2.1302, Training Accuracy: 22.80, 
Validation Loss: 2.9301, Validation Accuracy: 11.35
Epoch 3, Training Loss: 2.0950, Training Accuracy: 24.37, 
Validation Loss: 2.5193, Validation Accuracy: 10.35
Epoch 4, Training Loss: 2.0803, Training Accuracy: 25.45, 
Validation Loss: 2.4017, Validation Accuracy: 10.95
Epoch 5, Training Loss: 2.0561, Training Accuracy: 26.48, 
Validation Loss: 2.9083, Validation Accuracy: 10.05
Epoch 6, Training Loss: 2.0412, Training Accuracy: 27.35, 
Validation Loss: 2.6753, Validation Accuracy: 11.50
Epoch 7, Training Loss: 2.0220, Training Accuracy: 28.30, 
Validation Loss: 2.7593, Validation Accuracy: 10.85
Epoch 8, Training Loss: 2.0007, Training Accuracy: 28.67, 
Validation Loss: 2.6244, Validation Accuracy: 12.15
Epoch 9, Training Loss: 1.9894, Training Accuracy: 29.26, 
Validation Loss: 2.5236, Validation Accuracy: 12.10
E

train_accuracy,▁▄▅▆▆▇▇▇██
training_loss,█▅▄▄▃▃▂▂▁▁
val_accuracy,▆▅▂▄▁▆▄██▆
val_loss,▁█▃▂█▅▆▄▃▆
train_accuracy,29.66297
training_loss,1.97667
val_accuracy,11.6
val_loss,2.74002
